### Reference:

https://aws.amazon.com/blogs/machine-learning/elevating-the-generative-ai-experience-introducing-streaming-support-in-amazon-sagemaker-hosting/

In [9]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

Couldn't call 'get_role' to get Role ARN from role name username to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [10]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.1.0"
)

In [12]:
import json

In [33]:
model_id = 'teknium/OpenHermes-2.5-Mistral-7B'

In [114]:
instance_type = "ml.g5.2xlarge"
number_of_gpu = 1
config = {
    'HF_MODEL_ID': model_id, # model_id from hf.co/models
    'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
    'MAX_INPUT_LENGTH': json.dumps(15500),  # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(16384),  # Max length of the generation (including input text)
    'MAX_BATCH_TOTAL_TOKENS': json.dumps(16384),  # Limits the number of tokens that can be processed in parallel during the generation
}

In [115]:
llm_model = HuggingFaceModel(
    role=role,
    image_uri=llm_image,
    env=config
)

In [18]:
from datetime import datetime

In [116]:
endpoint_name = 'TGI-testing-openhermes-mistral-12-Jan-12-30-PM'

In [117]:
endpoint_name

'TGI-testing-openhermes-mistral-12-Jan-12-30-PM'

In [118]:
llm = llm_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=1200,
)

------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint TGI-testing-openhermes-mistral-12-Jan-12-30-PM: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [38]:
parameters = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.8,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.0,
    "stop": ["###", "</s>", tokenizer.eos_token],
}

In [40]:
prompt = '''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. We’ll name answer_not_found_in_context with the statement of ‘I am unable to find the exact answer for this’. If the answer is not present in the context, then respond by using answer_not_found_in_context. If the context has a related piece of information but not the exact answer, then follow up the answer_not_found_in_context with a concise relevant answer found from the context. Use three sentences maximum and keep the answer concise. It is extremely critical that you do not use any information which is not explicitly present in the context provided. 

for all queries with a time period reference, calculate the required time period using the following provided today's date in %d-%b-%Y format:

{today's date}: 11-Jan-2024

{question}: Sensex today


{context1}: Sensex Today | Stock Market LIVE Updates: Sensex off day's high, rise 100 points, Nifty above 21,600; auto index top sectoral gainer
Stock Market LIVE Updates: Tokyo's Nikkei closes above 35,000 for first time since 1990

Tokyo's Nikkei index closed above 35,000 for the first time in nearly 34 years on Thursday, buoyed by US tech rallies and hopes for an end to decades of sluggish inflation.The benchmark Nikkei 225 index rose 1.77 percent, or 608.14 points, to 35,049.86 -- the first time the index has risen above the 35,000 mark since February 1990 -- while the broader Topix index added 1.57 percent, or 38.39 points to 2,482.87.

Gold prices have been on an upward trajectory during 2023 and this momentum is expected to continue in 2024. Internationally, the volatility of the yellow metal prices will continue and will be triggered by various factors including the on-going geo-political scenario, strengthening of USD against INR and a pessimistic approach towards the growth of global economy.

Domestically, the demand for the yellow metal is expected to outgrow and is expected to touch the 70k mark on a domestic level. This rally will be driven by robust demand in the domestic market owing to the rising income levels and the inclination towards yellow metal. On the global level, we expect the prices to hover in the range of USD 1850-2050. Irrespective of the price movement, the consumer demand will continue to grow during the year backed by strengthened purchasing power, and the change in perspective towards gold as lifestyle statement, thus driving sales across along all ticket-sizes.

{context2}: Nifty 50, Sensex on January 11: What to expect in trade today
Ankit Gohel
The Indian stock market indices, Sensex and Nifty 50, are expected to open flat on Thursday amid positive global market cues. 

The trends on Gift Nifty also indicate a flat start for the Indian benchmark index. The Gift Nifty was trading around 21,716 level as compared to the Nifty futures’ previous close of 21,723.

Catch Live Market Updates here

On Wednesday, the domestic equity benchmark indices ended higher with the Nifty 50 taking the support near 20-day SMA (Simple Moving Average) and bouncing back sharply.

The Sensex rallied 271.50 points to close at 71,657.71, while the Nifty 50 settled 73.85 points, or 0.34%, higher at 21,618.70.

Nifty 50 formed a reasonable positive candle on the daily chart with lower shadow. Technically, this pattern indicates false downside breakout of the immediate support of 21,500 levels by an emergence of renewed buying from the lows.

Also Read: Indian stock market: 6 key things that changed for market overnight - Gift Nifty, US megacap stocks to bitcoin ETF nod

“After closing below the immediate support of 10-day EMA at 21,550 in the last couple of sessions, Nifty failed to show any sharp follow-through weakness/decisive downside breakout and has bounced back smartly from the lower levels. This is a positive indication," said Nagaraj Shetti, Senior Technical Research Analyst, HDFC Securities.

He believes the short-term trend of Nifty seems to have reversed up after a minor decline of the last two sessions. The Nifty is now expected to retest the upper trajectory around 21,750-21,850 levels in the next few sessions. 

(Exciting news! Mint is now on WhatsApp Channels Subscribe today by clicking the link and stay updated with the latest financial insights! Click here!)

Here’s what to expect from Nifty and Bank Nifty today:

Nifty 50 Predictions

The Nifty 50 index bounced back strongly from its important support at 21,500, on January 10, indicating that the bulls are quite active at this level. 

“The Nifty closed at 21,623, near its resistance. If it manages to close above 21,700-21,750 levels, we might see the Nifty reaching 22,000. For those looking to buy Nifty, the recommended range is between 21,550-21,600. Set a stop loss at 21,480 and aim for a target of 21,750," said 

Kunal Shah, Senior Technical & Derivative Analyst, LKP Securities.

Also Read: Buy or sell: Vaishali Parekh recommends three stocks to buy today — January 11

Bank Nifty Predictions 

The Bank Nifty index rebounded from day’s low and closed 118 points higher at 47,361 on January 10.

“The Bank Nifty index experienced a volatile trading session on the day of weekly expiry, but the bulls successfully defended the crucial support level of 47,000. Despite this, the overall market sentiment remains in “sell on rise" mode. The immediate hurdle for the index is at 47,500, where the highest open interest is concentrated on the call side," Shah said.

To initiate a short-covering move and potentially target the 48,000 mark, the index needs to convincingly surpass the level of 47,500 on a closing basis, he added.

Disclaimer: The views and recommendations made above are those of individual analysts or broking companies, and not of Mint. We advise investors to check with certified experts before taking any investment decisions.

Unlock a world of Benefits! From insightful newsletters to real-time stock tracking, breaking news and a personalized newsfeed – it's all here, just a click away! Login Now!

Catch all the Business News, Market News, Breaking News Events and Latest News Updates on Live Mint. Check all the latest action on Budget 2024 here. Download The Mint News App to get Daily Market Updates.
More
Published: 11 Jan 2024, 07:48 AM IST

Topics You May Be Interested In
{context3}: Stock Market Live Updates: Sensex gains 100 pts; Nifty above 21,650; Infosys dips 1% ahead of Q3 results
Updated - January 11, 2024 at 11:56 AM.

Sensex, Nifty, Share Prices LIVE: Benchmark equity indices are trading flat with positive bias amid a rally in global markets. As of 11:30 am, the Sensex gained 108.12 points to 71,765.83, while the Nifty rose 35.20 points to 21,653.90. Analysts anticipate intra-day volatility with Infosys and TCS announcing Q3 results. Infosys’ stock is under pressure as the company is set to announce the financial results for Q3 FY24 post the market hours later today. Global markets show strength. SBI Capital expects CY24 to address economic aftermath, prioritising growth over inflation. Market advice includes booking profits in sectors with elevated valuations. Pharma remains attractive, and the chemical sector shows signs of recovery. Overall, markets are cautiously optimistic, closely monitoring Q3 results and potential shifts in 2024 trends.


Stock Market today | Share Market Live Updates - Find here all the live updates related to Sensex, Nifty, BSE, NSE, share prices and Indian stock markets for 11 January 2024.

January 11, 2024 11:56
Stock Market Live Updates: India Japan Fund to invest ₹400 crore in Mahindra Last Mile Mobility at a valuation of ₹6000 crore

January 11, 2024 11:41
Stocks Market Live Updates: Centrum Group appoints Manish Jain as Head of the Fund Management businesses

Centrum Group today announced the appointment of Manish Jain as Head of the Fund Management businesses. He will lead and manage Centrum’s Portfolio Management Services (PMS) and Non-discretionary Equity Advisory verticals. 

Manish comes with a proven track record of over 20 years in the Indian equities market, managing equity funds, driving high quality equity research and with expertise both on the buy and sell sides. He has held senior positions with leading financial institutions such as Nomura / Lehman Brothers, ICICI Securities, and most recently with Ambit Investment Advisors, where he was managing their Coffee Can Portfolio having an AUM of approx. Rs 1,150 crores. Manish is a Chartered Financial Analyst (CFA) and a MBA in Finance from Symbiosis, Pune.

January 11, 2024 11:33
Stock Market Live Updates: Discount brokers hold 65% share, MTF achieves a new high: ICRA

ICRA has analysed the performance of 29 prominent brokerage companies in H1 FY2024. Supported by improved market activity, ICRA’s sample set of companies reported a sharp 20% annualised growth in the net operating income. Given the benefits of economies of scale, the aggregate profitability of this sample set also improved, with a 24% annualised increase in the net profit and a return on equity of 27% in H1 FY2024 compared to 25% in FY2023. ICRA expects the industry to report all-time high revenues and net profits with a year-on-year (YoY) expansion of 18-22% and 22-25%, respectively, in FY2024. 

After reporting a flattish trajectory in FY2023, the aggregate industry-wide MTF exposure grew by 98% from March 2023 to ~Rs. 51,000 crore on December 28, 2023. 

The participation of retail investors also remained strong in the futures & options segment, driven by the launch of new index options, miniaturisation of contracts/lot sizes, and separate weekly expiries for each index option. During FY2020 to FY2023, the number of contracts traded increased by about nine times. Moreover, the number of contracts traded expanded by 2.4 times, on a YoY basis, in H1 FY2024. 

Given the substantial new client additions in the past few fiscals, discount brokerage houses continue to hold a leadership position, in terms of market share of active NSE clients, at 65% in November 2023 compared to 18% in March 2019. The market share of non-bank full-service brokers and bank-based brokers stood at ~21% and 14%, respectively, in November 2023.

Although the borrowings by brokerage entities were historically low, financial leverage has increased considerably in recent fiscals with the material scale-up of the MTF and increased working capital requirements due to regulatory changes for the securities broking industry. 

The aggregate leverage of bank brokers, which dominate the MTF segment, increased to 1.9 times as of September 30, 2023, from 0.4 times as of March 31, 2020. During this period, the aggregate gearing of non-bank full-service brokers increased to 1.1 times from 0.4 times. Discount brokers, with a limited presence in MTF, reported marginal dependence on borrowings.

January 11, 2024 11:21
Financial Market Live Updates: We deep-dive into business models of regulated entities to spot ALM mismatches, assess funding stability, and liquidity position: RBI Guv Das

January 11, 2024 11:20
Financial Market Live Updates: Indian banking system is well-placed to support India’s growth story in the years ahead: RBI Guv Das.

January 11, 2024 11:19
Stock Market Live Today: NIACL receives SCN cum demand notice from Directorate General of GST for Tax of Rs 2,379 crore.

January 11, 2024 11:18
Stock Market Live Today: Polycab: Income Tax Department detects unaccounted cash sales of about Rs 1,000 crore after it initiates search.

January 11, 2024 11:18
Stock Market Live Today: Metro Brands: Deepika Deepti has resigned as Senior Vice President- Marketing of the company with effect from March 26.

January 11, 2024 11:17
Stock Market Live Updates: Bank of Baroda hikes lending rates by 5 bps for overnight, six month & one year tenures.

January 11, 2024 11:17
Stock Market Live Updates: Mahindra & Mahindra’s stake in Sustainable Energy Infra Trust diluted to 10.5% from 15.7% earlier.

January 11, 2024 11:15
Stock Market Live Updates: ONGC incorporates wholly owned subsidiary OVL Overseas IFSC in GIFT City, Gujarat.

January 11, 2024 11:15
Stock Market Live Updates: Satin Creditcare to consider Fund Raising via NCDs on January 15.

January 11, 2024 11:14
Stock Market Live Updates: Safari Industries’ Board of directors will meet on January 15 to consider raising funds.

January 11, 2024 11:12
Stock Market Live Today: Thirumalai Chemicals has started commercial production at its subsidiary, TCL Intermediates Private Limited’s Greenfield plant in Dahej.

January 11, 2024 11:11
Stock Market Live Today: Senco Gold achieves 9% volume growth in gold and 27% in diamond jewelry.

January 11, 2024 11:11
Stock Market Live Today: KIMS has acquired an additional 5.04% equity stake in Sarvejana Healthcare Private Limited.

January 11, 2024 11:10
Stock Market Live Today: Kalyani Steels secures acquisition of Kamineni Steel & Power India Pvt Ltd assets under the Insolvency and Bankruptcy Code, 2016.

January 11, 2024 11:10
Financial Market Live Updates: RBI supervisory process made more dynamic to “smell possible distress early” in regulated entities: RBI Guv Das.

January 11, 2024 11:09
Stock Market Live Updates: Ajmera Realty & Infra reports 63% YoY increase in sales area in Q3 FY24

Ajmera Realty & Infra India Ltd reported robust growth in Q3 FY24 with a significant 63% YoY increase in sales area, at 1,03,573 sq.ft, and an equivalent sales value of Rs 253 crores. Despite limited inventory, growth in sales value came on the back of sustained interest in key projects, Ajmera Manhattan and Ajmera Eden in Mumbai, as well as projects in Bengaluru.

January 11, 2024 11:04
Stock Market Live Updates: SBI Securities Daily Market Pulse 11 January 2024

● From Price Action Perspective:

Led by a strong move in Reliance & ICICIBANK, Nifty took support at 21450 zone & witnessed a strong rebound towards 21620 zone, gaining 0.34% while Banknifty gained 0.25 % yesterday rebounding from crucial support zone of 47000

● Decoding Asset Relationships:

The US Dollar Index is currently trading around 102.50 levels while US 10-year yields are quoting at 4%, signaling a probable delay in the Interest Rate Cuts by the Fed.

Brent Crude remains steady between $75-78 a barrel.

Below 75$, crude prices could weaken upto $70-71 a barrel, which would be a positive factor from a Macro-economic standpoint.

● Quick Check on Global Markets:

The S&P 500 has taken support around its 20-EMA around 4680-4700 and is now trending towards its life highs. Looking ahead, till the 20-day EMA holds, the index can continue its positive momentum towards 4840-4890.

On the downside, 4680-4700 zone could act as important support.

● Key Nifty levels:

Going forward, support is likely to be seen at 21470-21500 level from which the index has bounced five times in the last fortnight while, on the upside, immediate resistance is at 21710-21730.

On a close above 21730, Index would inch towards a fresh life high of 21900-21950.

In case 21480 is breached, Index could slide towards 21370-21380 zone where 20 Day EMA is placed.

Upcoming Quarterly Results by the IT Companies would act as a key trigger going forward.

However, outperformance by Reliance is a big positive for the markets & would provide support to Nifty on dips.

● OI data Interpretation for weekly series,

Significant call Writing buildup is witnessed in 21650 and 21700 strikes, while on the puts front, 21600-21500 witnessed considerable Open Interest additions.

Overall, the range for the index could be 21450 on the downside & 21750 on the upside.

● Key Bank Nifty Levels:

Going forward, the zone of 47000-47130 is likely to act as a support while, on the upside, resistance is seen at 47600-47700 zone.

Above 47700, Bank Nifty could witness sharp short-covering upto 47950-48000.

In case 47000 breaks, Index can retest its next support placed at 46800-46650.

For the next weekly Expiry, significant Open Interest buildup is witnessed in 47500-47700 calls and 47000 puts, implying a near-term range of 47700 and 47000 for the Banking Index.

● Sectors & stocks in Momentum:

The most influential stocks whose results are to be released today are Tcs, Infosys & Hdfcamc.

Railway stocks such as IRFC, IRCTC, RVNL, and Rites experienced buying with heavy volumes yesterday.

From the broader market, bullish setups were observed in Reliance, Adaniports, ApolloHosp, Delhivery, Tiindia, Crompton, Maxhealth, Jswenergy, Prestige, Bhel, Voltas, Trent, Ltts, Abbotindia, Poonawalla, Persistent, Tv18brdcst, Tanla, Mazdock, Centurytex, Amber, Bsoft, Nbcc, Gnfc.

January 11, 2024 11:02
Stock Market Live Updates: Government initiates process to sell enemy shares in Peerless, Mohan Meakin and 18 others

The Government has decided to sell enemy shares of 20 companies that are being held by the Custodian of Enemy Property of India (CEPI).

According to a notice by the Finance Ministry, the list of companies includes, Anand Projects Limted, Bharat Nidhi Limited, Delhi Flour Mills, Mohan Meakin, Spencer & Co, the Peerless General Finance & Investment, the Bengal Dooars National Tea Limited, and Woodbriar Estate beside others. SBI Caps is the merchant banker and selling broker for the transaction.

January 11, 2024 11:01
Bank Nifty prediction today–Jan 11, 2024: Index might consolidate today

Bank Nifty opened today’s session with a gap-up at 47,538.60 versus yesterday’s close of 47,360.85. The index is now at 47,500, up 0.3 per cent.

The advance/decline ratio stands at 9/3. Bandhan Bank, up 2 per cent, is the top gainer whereas Federal Bank, down 0.3 per cent, is the top loser.

January 11, 2024 10:59
Nifty prediction today – Jan 11, 2023: Index remains in a range, stay out for now

Nifty 50 began today’s session with a gap-up at 21,688 versus yesterday’s close of 21,618.70. The index is now at 21,660, up 0.2 per cent.

Supporting the bullish bias, the advance/decline ratio of Nifty 50 stands at 33/17. However, the index remains within the range it has been trading for the past couple of weeks.

January 11, 2024 10:58
Financial Market Live Updates: Indian banking sector has emerged stronger: RBI Guv Das

Reserve Bank of India Governor Shaktikanta Das has stated that all key indicators of scheduled commercial banks have demonstrated improvement over the past four years. Governor Das further noted the enhanced financial indicators of Non-Banking Financial Companies (NBFCs) during this period. 

Highlighting the central bank’s proactive measures, Das mentioned that since 2020, the RBI has undertaken a comprehensive overhaul of the regulatory architecture, incorporating significant changes such as the large exposure framework and scale-based regulation for NBFCs. 

Flow of Investments in fintech space, especially digital lending, has gone up. Regulatory architecture gives confidence to all stakeholders, including foreign investors.

January 11, 2024 10:52
Stock Market Live Today: CESC’s unit, Noida Power Company, has received an order from the Uttar Pradesh Electricity Regulatory Commission.

January 11, 2024 10:52
Stock Market Live Today: Atul Products Ltd, has received approval and started operations of a 300 tpd caustic and 50 MW power plant.

January 11, 2024 10:51
Stock Market Live Today: Welspun Enterprises pledges Rs 40,000 crore investment to set up green hydrogen, green ammonia ecosystem in Gujarat.

January 11, 2024 10:50
Stock Market Live Today: Maruti Suzuki plans to produce 10 lakh units annually at new plant.

January 11, 2024 10:50
Stock Market Live Today: NBCC bags contract worth Rs 218 crore from NIFT campus at Varanasi, Uttar Pradesh.

January 11, 2024 10:48
Stock Market Live Today: Ease My Trip ventures into insurance sector with the launch of its subsidiary.

January 11, 2024 10:46
Stock Market Live Today: Steel Strips: Directorate General Of Trade Remedies recommends anti-dumping duty on aluminum alloy road Wheel Imports from China for 5 years.

January 11, 2024 10:46
Stock Market Live Today: Sairam Prasad named as CEO of unit Suzlon Global Services effective January 16.

January 11, 2024 10:45
Stock Market Live Today: NCC: Smallcap World Fund INC bought 36.15 Lakh Shares at 185.23/share.

January 11, 2024 10:43
Stock Market Live Today: Lupin launches new product in the U.S. market designed to aid smoking cessation treatment.

January 11, 2024 10:41
Stock Market Live Today: Phoenix Mills reports a significant year-on-year increase of 30%, with gross retail collections reaching Rs 700 crore.

January 11, 2024 10:23
Commodity Market Live Updates: Crude oil gains despite surprise rise in US inventory

Despite a surprise increase in crude oil inventories in the US, crude oil futures traded higher on Thursday morning as the Houthis continued their attacks on merchant vessels in the Red Sea region on Tuesday.

At 9.54 am on Thursday, March Brent oil futures were at $77.09, up by 0.38 per cent, and February crude oil futures on WTI (West Texas Intermediate) were at $71.63, up by 0.36 per cent.

January 11, 2024 10:13
Stock Market Live Updates: Infosys under pressure ahead of Q3 FY24 results

The stock price of Infosys declined 1.44 per cent or ₹21.95 to trade at ₹1,497.70 at 10 am on BSE. The IT major was the major laggard within the 30-stock Sensex pack in early trade.

Infosys’ stock is under pressure as the company is set to announce the financial results for Q3 FY24 post the market hours later today.

January 11, 2024 10:09
Stock Market Live Updates: Markets climb in early trade on firm global trends

Benchmark equity indices climbed in early trade on Thursday amid a rally in global markets and buying in Reliance Industries.

The 30-share BSE Sensex jumped 341.76 points to 71,999.47 in early trade. The Nifty climbed 107.8 points to 21,726.50.

January 11, 2024 10:00
Currency Market Live Updates: Rupee gains 2 paise to 83.01 against US dollar

The rupee stayed firm for the seventh consecutive session and gained 2 paise to 83.01 against the US dollar in early trade on Thursday boosted by a weak American currency and softening crude oil prices.

Sustained buying in the domestic equity markets also supported the Indian currency even as investors were awaiting US inflation data to be released later in the day, forex traders said.

January 11, 2024 09:24
Stock Market Live Today: 360 One Prime NCDs offer up to 9.65% yield: Should you invest?

The trend of newer companies tapping the bond market to raise funds directly from retail investors has been a prominent development over the past year. Interest rates are peaking globally, and yields have stabilised after surging to record levels. The Reserve Bank of India (RBI) and the US Federal Reserve have hit the pause button for a while now.

January 11, 2024 09:22
Stock Market Live Today: SBI raises $600 million via 5-year senior unsecured fixed rate notes

SBI concludes issue of $600 million fixed rate 5-year notes at coupon of 5%. The instrument is being listed at SGX and India International Exchange

January 11, 2024 09:21
Stock Market Live Today: SpiceJet board approves issue of 31.8 crore shares at Rs 50/share on preferential basis:

January 11, 2024 09:20
Stock Market Live Today: Bulk Deal & Insider Trade Update

Bulk Deals

*Ncc Limited : Smallcap World Fund Inc Buy 3615365 Shares @ Rs. 185.23 

*Vl E Gov And It Sol Ltd : Ag Dynamic Funds Limited Buy 600000 Shares @ Rs. 69.93

*Goyal Salt Limited : Tryrock Capital Trust I Sell 192000 Shares @ Rs. 

Insider Trade

Acquisition

* Anshuman Kedia Revised Promoter & Director Acquisition 24200 shares of Associated Alcohols & Breweries Ltd. on 28-Sep-23

* Vedant Kedia Revised Promoter Group Acquisition 24200 shares of Associated Alcohols & Breweries Ltd. on 28-Sep-23

Pledge 

* PENNAR HOLDINGS PRIVATE LIMITED Revised Promoter Pledge 1500000 shares of Pennar Industries Ltd.-$ on 18-Sep-23

Dividend Update

--

Result Today

Tata Consultancy Services Ltd.

Infosys Ltd.

HDFC Asset Management Company Ltd.

GTPL Hathway Ltd.

5Paisa Capital Ltd.

Rajoo Engineers Ltd.

AGI Infra Ltd.

Plastiblends India Ltd.

January 11, 2024 09:19
Commodities Market Live Today: Crude oil rises on renewed Red Sea tensions; Brent at $77.12, WTI at $71.64

Crude oil futures traded higher on Thursday morning as the Houthis continued their attacks on merchant vessels in the Red Sea region on Wednesday. At 9.13 am on Thursday, March Brent oil futures were at $77.12, up by 0.42 per cent; and February crude oil futures on WTI (West Texas Intermediate) were at $71.64, up by 0.38 per cent. January crude oil futures were trading at ₹5962 on Multi Commodity Exchange (MCX) in the initial trading hour of Thursday morning against the previous close of ₹5931, up by 0.52 per cent; and February futures were trading at ₹5991 as against the previous close of ₹5957, up by 0.57 per cent.

January 11, 2024 09:14
Stock Market Live Today: The Baker’s Dozen bags ₹33 crore in funding round led by Wipro Consumer’s VC arm

Artisanal bakery brand, the Baker’s Dozen has secured ₹33 crore in a pre-series A funding round led by Wipro Consumer Care-Ventures. Fireside Ventures, Mirabilis Investment Trust and She Capital also participated in this round.

Sneh Jain, Co-Founder & Managing Director, The Baker’s Dozen told businessline, “ Nearly 70-80 per cent of our sales come from the online channel and almost 75 per cent of our sales come from the three cities of Delhi, Mumbai and Bengaluru. The funds will help us deepen our presence in other metros as well as widen our geographical expansion to tier-1 cities. We are also looking to strengthen our offline presence and will also focus on ramping up strategic marketing and brand-building initiatives.”

January 11, 2024 09:13
Stock Market Live Today: Geojit Financial Services’ V K Vijayakumar: FII and DII actions limit market, Q3 earnings awaited

V K Vijayakumar, Chief Investment Strategist, Geojit Financial Services.

“There is no consistency in FII and DII behaviour so far this month. They are doing alternative bouts of buying and selling which is restricting the market in a range. The market needs triggers to break out or breakdown from this range. A probable negative trigger can be a slightly hawkish statement from the Fed postponing the rate cuts which the market expects to begin by March 2023. The US CPI inflation data due tonight will provide cues regarding this.

The Q3 results season starting today with the results of TCS and Infy will provide indications of the Nifty earnings for FY24. Financials, capital goods, telecom, automobiles and hotels will post good numbers. IT results will be tepid and FMCG will be a mixed bag.

More than broad market action, market responses will be stock-specific in response to results and management commentary.”

January 11, 2024 09:12
Stock Market Live Today: SMC Global: Market set to open flat as investors await IT giants’ Q3 results

The market is expected to open on a flat note today. Investors await the release of Q3 results from major IT companies TCS and Infosys later today, marking the beginning of the earnings season for the December quarter. Asian stock markets saw an uptick, following the trend in US stocks as investors awaited inflation data set to be released on Thursday. Bitcoin surged following the U.S. regulators’ approval of exchange-traded funds linked to the cryptocurrency. U.S. stocks closed higher on Wednesday as megacaps rallied, but gains were limited ahead of inflation reports and major bank earnings later in the week. Oil steadied on Wednesday, giving up most of its earlier gains, as Middle East supply concerns arising from the Israel-Hamas war and the shutdown of a top Libyan oilfield balanced rising U.S. output and worries about weak economic growth. As per provisional figures,

Foreign Institutional Investors (FIIs)/ Foreign Portfolio Investors (FPIs) sold shares worth net Rs.1721.35 crore and Domestic institutional investors bought shares worth net Rs.2080.01 Crore on 10th January 2024.

January 11, 2024 09:10
Stock Market Live Today: Morning Outlook Quote:  Deepak Jasani, Head of Retail Research, HDFC Securities. (January 11)

Indian markets could open flat, despite mildly higher Asian markets today and positive US markets on Jan 10

U.S. stocks finished higher on Wednesday, a day ahead of a widely anticipated December inflation report, though the S&P 500 fell short of an all-time high after remarks from a key Federal Reserve policymaker.

Inflation, along with its impact on bond markets and the Federal Reserve’s monetary policy trajectory, is the primary issue at hand this week as investors await Thursday’s consumer-price index reading for December. Economists expect annual headline CPI inflation to inch up to 3.2% last month from 3.1% in November. The core reading, which strips out more volatile items like food and energy, is expected to fall to 3.8% year-over-year, from 4% previously. In U.S. economic data, wholesale inventories declined 0.2% in November, according to the Commerce Department. The Securities and Exchange Commission greenlighted several spot bitcoin exchange-traded funds for the first time on Wednesday, according to a filing posted on the U.S. regulatory agency’s website.

The move paves the way for a new class of investors to gain exposure to the virtual currency and could serve as a tailwind for the price of the world’s most popular digital asset. Bitcoin was mostly flat at around $46,742 following the SEC news, according to CoinDesk data. Asian stocks and US and European equity futures rose ahead of inflation data due later Thursday that will help clarify the path ahead for Federal Reserve policy. Nifty extended gains from Tuesday and ended higher on Jan 10 after a volatile session as Reliance Industries rose to hit its lifetime high level. At close, Nifty was up 0.34% or 73.9 points at 21618.7. The short-term trend of Nifty seems to have reversed after the recent minor decline. The Nifty is now expected to retest the upper trajectory around 21765-21834 levels in the next few sessions. Immediate support is placed at 21449 levels.

January 11, 2024 09:09
Stock Market Live Today: Kotak Securities’ Shrikant Chouhan: Nifty witnesses bullish move, support at 20-day SMA, eyes 21680-21725 levels

Shrikant Chouhan, Head – Equity Research, Kotak Securities

The benchmark indices witnessed a bullish move on Wednesday, with Nifty closing 74 points higher while Sensex was up 272 points. The media index rose over 3%, while intraday profit booking was seen in select PSU banks and FMCG stocks. Technically, after morning intraday correction, Nifty took support near the 20-day SMA (Simple Moving Average) and made a bullish comeback. From the day’s lowest point, the market came up by more than 190/660 points. Traders should consider the 20-day SMA or 21500/71250 as a crucial support level. Above this, the pullback formation is likely to continue. The market has prospects of reaching 21680-21725/71900-72050. However, selling pressure is likely to increase below the 20-day SMA or 21500/71250. If the market slips below that, it may fall to 21400-21355/71000-70800.

Traders should take profit on long positions or reduce weak long positions between 21680/21720 levels. For the bank nifty, 47000-46900 would be the ultimate support zone and if it falls below that, it could reach 46500 levels. On the higher side, 47550 is the biggest hurdle, so traders should consider reducing positions around the same level.

January 11, 2024 09:09
Stock Market Live Today: Mehta Equities’ Prashanth Tapse analyses 2024’s volatile market: Focus on US CPI, TCS, and Infy earnings

-- Prashanth Tapse, Senior VP (Research), Mehta Equities Ltd

“In the current volatile market of 2024, marked by conflicting trends, uncertainties around US interest rates and India Inc.’s corporate earnings drive disagreement. Key factors today include the eagerly awaited US CPI release and the commencement of Q3 earnings by TCS and INFY. The Federal Reserve’s interest rate decisions remain central, with upcoming CPI and PPI indicators influencing their stance. INFY or TCS profit guidance could sway sentiments. Technically, Nifty finds vulnerability below 21307, confirming strength above 21836. Nifty options suggest a 21300-22000 trading range. Preferred trades for Nifty and Bank Nifty are provided. The chart of the day is bullish on select stocks, with a standout recommendation to buy ITD Cementation at an entry price of 300.75, targeting 600, with a stop loss at 189, for a holding period of 0-365 days.”

January 11, 2024 09:08
Commodities Market Live Today: Crude oil sees volatility amid U.S. inventory surge

“Crude oil exhibited significant volatility, experiencing a sharp decline amid the increase in U.S. oil inventories and in anticipation of the U.S. inflation data. U.S. crude oil inventories surged by 1.3 million barrels for the week ending on January 5, contrary to expectations of a 0.7 million barrel decline. The struggle in crude oil prices is attributed to the upsurge in U.S. oil stocks and a stable dollar index. Key U.S. inflation data, set to be released later today, could provide further guidance for oil prices. We anticipate that crude oil prices will continue to be volatile in today’s trading session. Support for crude oil is expected at $70.80–70.10, with resistance at $72.20-73.00 during today’s session. In Indian Rupees (INR), crude oil has support at Rs 5,850-5,770 and resistance at Rs 6,010-6,090.” -- Rahul Kalantri, VP Commodities, Mehta Equities Ltd

January 11, 2024 09:06
Commodities Market Live Today: Gold and silver show vulnerability amidst U.S. inflation data anticipation; dollar index stability prevails

“Gold and silver traded within a narrow range, displaying some vulnerability ahead of the impending U.S. inflation data. The dollar index maintains stability above the 102 marks following the release of minutes from the U.S. Federal Reserve monetary policy meeting and encouraging U.S. job data. The uncertainty surrounding the timing of potential Fed rate cuts exerted downward pressure on gold and silver prices in the global markets, reinforcing the strength of the dollar index. However, ongoing tensions in the Red Sea and the Middle East are providing some support at lower price levels. Anticipating today’s session, we forecast continued volatility in gold and silver prices. Gold finds support at $2016-2002, with resistance at $2041-2055. Meanwhile, silver’s support is identified at $22.72-22.55, and resistance is positioned at $23.20-23.38. In Indian Rupees, gold is anticipated to find support at Rs61,750-61,510, with resistance at Rs62,290 and Rs62,510. Silver’s support in INR is projected at Rs71,550-71,080, with resistance at Rs72,650 and Rs73,180.” -- Rahul Kalantri, VP Commodities, Mehta Equities Ltd

January 11, 2024 09:03
Stock Market Live Today: Pre-market views by Avdhut Bagkar of StoxBox

The US market closed higher on Wednesday as megacaps rallied, but gains were limited ahead of inflation reports and significant bank earnings later in the week. The Labor Department’s reports on consumer and producer price inflation, which are due to be released on Thursday and Friday, respectively, could significantly impact the outlook for interest rates. Asian markets experienced a notable upswing on Thursday, following the positive momentum in US stocks ahead of the release of inflation data. Japan’s Nikkei extended its record-breaking rally on dovish BOJ bets. Indian market is seen opening flat to slightly higher on Thursday as investors await earnings from IT majors as well as the latest readings on industrial output and retail inflation for directional cues. Tata Consultancy Services and Infosys are set to kickstart Q3 FY2024 earnings on January 11. We expect TCS revenue to be flat sequentially as growth was tepid due to ongoing weaknesses in discretionary spending and layoffs during the quarter. At the same time, Infosys revenue marginally declined QoQ owing to a challenging quarter primarily influenced by sustained weak demand, the absence of certain one-time sales and the impact of layoffs.

The 50 index witnessed a volatile opening and traded with a negative bias until the mid-trading session and marked the day’s low at 21468. However, the index saw a quick short recovery from the support zone near 21500-21450 allowing the index to close in its upper quartile of the trading range. On the derivative front, 21500PE added the highest open interest indicating a strong support zone and sustenance above this zone will bolster the price action to garner bullish strength.

January 11, 2024 08:47
Stock Market Live Today: Parag Parikh Conservative Hybrid: Ideal fund for investing at market highs

While all eyes are on the stock indices scaling new lifetime highs, the bond market also offers attractive opportunities. Yields on commercial paper and bonds from highly rated corporates are ruling at near 8 per cent levels, with credit offtake picking up. Conservative Hybrid Funds make for a good investment for investors who are bullish about bonds but wary of equity risks.

January 11, 2024 08:43
Stock Market Live Today: Tembo Global exceeds Rs. 300 crore Q3 turnover

Tembo Global Industries Limited has reveald its outstanding achievement of surpassing a turnover of over Rs. 300 crores in the third quarter ending on 31st December 2023, for the financial year 2023-24. 

They recorded a turnover of Rs. 249 crores for the year ending on 31st March 2023 and aim to achieve more than Rs. 375 crores in the current year, representing an approximate 50% increment in revenue.

January 11, 2024 08:42
Commodities Market Live Today: Thermal coal prices at 2.5-year low - Elara Securities

Global crude steel production up ~3% YoY in November

The World Steel Association (WSA) data shows global crude steel production sustained growth for the second consecutive month, expanding ~3% YoY to 145.5mn tonne in November. This growth primarily was propelled by ~7% YoY rise in production for the rest of the world (ROW) while China’s showed a steady trend with no significant YoY change. However, on a MoM basis, production failed to show any upward movement for another month and contracted ~3%, dragged by a MoM fall of ~4% and ~3% in China and ROW, respectively. In November, India’s crude steel production rose ~11% YoY but dropped ~4% MoM. As per provisional data, India’s crude steel production rose ~8% YoY and ~2% MoM in December.

January 11, 2024 08:42
Stock Market Live Today: Sectoral Recommendations: Power generation sustains momentum; Earnings to be resilient on robust demand - Elara Securities

Power generation remained buoyant in Q3FY24, led by surging demand amidst festive season. Generation rose 13% YoY to 416BUs in Q3FY24, albeit on a high base of last fiscal (up 10% in FY23). Generation rose a substantial 25% YoY to 150BUs in October, which was characterized by high humidity. November saw an 11% YoY increase in generation to 130BUs, aided by uptick in economic activity amidst festival season. Generation moderated in December, up just 4.5% YoY to 134BU. Peak demand moderated to 222GW in Q3FY24 from a record high of 243GW in H1FY24 due to reduced cooling demand.

January 11, 2024 08:41
Stock Market Live Today: Stock Recommendations: BPCL | Unfavourable risk-reward - HDFC Securities

Bharat Petroleum Corporation (BPCL) stock has rallied ~35% and has outperformed the Sensex by ~26% over the last 3M, driven by improving auto fuel marketing margins as Brent crude price declined by ~20% to USD 76/bbl over the same period and strong refining margins as diesel and jet fuel crack spreads averaged USD 23-24/bbl. We downgrade BPCL to a REDUCE from a BUY, with a target price of INR 475/sh, owing to the risk of (1) lower auto-fuel marketing margins because of either a bounce-back in crude oil price or retail price cuts ahead of general elections and (2) moderation in refining margins.

January 11, 2024 08:41
Stock Market Live Today: Stock Recommendations: V-MART Retail | An under-earning asset or a melting ice cube? - HDFC Securities

After going through a challenging period marked by (1) an unfavourable rural cycle, (2) deteriorating unit economics, and (3) stumbling in assortment pricing and capital allocation, VMART’s KPIs are bottoming out. Notable improvements include (a) gradual recovery in footfall/sales densities, (b) downward recalibration of rental bills, and (c) alleviation of working capital challenges. We recently interacted with V-MART’s management to better understand the levers to normalcy. Key takeaways include (1) a healthy festive season with 4/10% SSSG/revenue growth in Q3; (2) streamlining of the buying/merchandising process that will aid sourcing margins which could be passed on to the end-consumer to earn higher sales/sq ft and improve WC efficiency; (3) successful performance of new ‘Unlimited’ stores at >INR8,000/sq. ft; and (4) a significant paring of Limeroad losses expected in FY25. Rising competitive intensity concerns seem a false positive as store overlap of major value retailers remains low. Against this backdrop, V-MART resembles a momentarily under-earning asset (awaiting a turn in the rural cycle) more than a melting ice cube. Risk-reward seems favourable @ <20x FY26 EV/EBITDA. Our base case for core VMART bakes in ample margin of safety (FY23-26 sales/sq ft CAGR: 3%; @ INR7.4k and 5.5/6.7% EBITDAM for FY25/26 vs 9.3% in FY19). We upgrade the stock to a BUY with a DCF-based TP of INR2,550/sh (earlier 1,850/sh); implying 23x FY26 EV/EBITDA (FY25/26 EBITDA changes by +21/+7% respectively).

January 11, 2024 08:40
Stock Market Live Today: Sectoral Recommendations: IT Sector | Weak quarter; focus on CY24: HDFC Securities

The IT sector (coverage universe) is expected to post a weak quarter with a sequential decline. This is due to increased furloughs, Q3 seasonality, a prolonged deal conversion cycle, and compression in discretionary and renewals. Positive factors for the quarter include the ramp-up of mega deals (BSNL/Verizon for TCS/HCLT) and consistent deal activity. However, most of the supply-side indicators, such as tech job postings, remain soft and do not signal any recovery ‘yet’. Although the recent trajectory of rates can support valuation multiples (high inverse correlation), it’s unlikely that tech budgets will improve materially. This is because a ‘slowdown’ in economic growth remains a baseline scenario, leading enterprises to prioritise cost optimization over transformation initiatives. In the Indian IT landscape, we expect three developments: (1) growth divergence within the sector in the near term, (2) margin recovery with favourable supply-side factors, and (3) mid-tier IT sustaining its relative outperformance. However, the margin of safety is low on aggregate, as valuations have approached +2 SD (rerated higher by >20% in the last six months) with limited immediate upside triggers. We maintain our preference for LTIMindtree, Persistent, and Birlasoft.

January 11, 2024 08:40
Stock Market Live Today: Sectoral Recommendations: BFSI | Emerging long-tail risks in unsecured credit - HDFC Securities

India’s heterogenous unsecured retail credit market (US$160bn; 3-year CAGR at ~23%) has been in the headlines ever since the RBI clamped down on select categories of consumer credit. Our discussions with experts suggest that the RBI’s note of caution is largely centred on small-ticket personal loans (STPL) (ticket size <INR100k), which account for a high incidence of systemic blind spots, dominant share of incremental disbursements, and elevated early-stage delinquencies, especially across static pools. The RBI’s risk-weight measures are likely to increase the hurdle rate for banks and NBFCs towards unsecured consumer loans, resulting in a material growth deceleration. We continue to hold our below-consensus growth forecasts (1-2 percentage points across our coverage universe) and remain conservative on incremental spreads (blended FY24 NIMs at 15-20bps below FY23 levels), considering the impact on incremental asset mix. Our top picks are ICICIBC (TP: INR1,200) and SBIN (TP: INR780) among large banks; and BAF (TP: INR8650) and SBICARD (TP: INR955) among NBFCs

January 11, 2024 08:40
Stock Market Live Today: Sectoral Recommendations: Retail by Emkay Global

Larger-ticket segments lead growth; AVL, Senco, GO key outliers

Discretionary trends remain mixed, with muted growth in low-ticket categories like QSR/Apparel and strong growth in high-ticket categories like Jewelry /Durables/Luxury. Within Jewelry, Senco (Buy) has delivered 24% top-line growth (vs. our expectation of 15% growth), driving 5-8% increase in our estimate/TP. For TTAN, domestic jewelry growth at 21% is 4-5% above expectations, but a likely lower studded mix limits the EPS upgrade. We expect durables retailer AVL (Buy) to log the best topline growth among our Retail universe, at ~35%, aided by strong positioning in the fast-growing Hindi-Heartland. Valuations are also supportive, at ~40x FY25 EPS. Go Fashion (Buy) will continue outperforming, with 15% topline growth vs. flattish organic growth for ABFRL/PAG. WC optimization/closure of pledge are added triggers. Within QSRs, we favor Westlife on better SSG confidence and Sapphire on potential re-rating. Compelled either by muted trends or recent run-ups, we have a Reduce or Sell on TTAN, ABFRL, PAG, JUBI, DEVYANI, VBL and ETHOS.

January 11, 2024 08:39
Stock Market Live Today: HLE Glascoat: The company plans to acquire a 70% stake in Kinam Engineering Industries, holding 35.56% shares in KEIPL.

January 11, 2024 08:39
Stock Recommendations: Fund Houses Recommendations  

Jefferies on Bharti Airtel: Maintain Buy on Company, raise target price at Rs 1300/sh 

MS on Bank India: Maintain Overweight on Bank, target price at Rs 135/sh 

MS on Coforge: Initiate Overweight on Company, target price at Rs 7200/sh 

MS on Manappuram: Maintain Overweight on Company, target price at Rs 210/sh 

HSBC on Zomato: Maintain Buy on Company, raise target price at Rs 150/sh 

Nuvama on Vedanta: Upgrade to Buy on Company, raise target price at Rs 362/sh 

Emkay on Ultratech: Maintain Buy on Company, target price at Rs 11400/sh 

Emkay on JK Cement: Maintain Add on Company, target price at Rs 310/sh 

Emkay on Sagar Cement: Maintain Add on Company, target price at Rs 4260/sh 

MOSL on Metro Brands: Maintain Buy on Company, target price at Rs 1530/sh 

Jefferies on Indus Tower: Maintain Neutral on Company, raise target price at Rs 160/sh 

Kotak on Jubilant Food: Maintain Reduce on Company, target price at Rs 500/sh 

Kotak on Devyani: Maintain Add on Company, target price at Rs 110/sh 

Kotak on Restaurants Brand: Maintain Reduce on Company, target price at Rs 110/sh 

Kotak on Sapphire: Maintain Buy on Company, cut target price at Rs 1640/sh 

Kotak on Westlife: Maintain Reduce on Company, cut target price at Rs 800/sh 

Investec on HCL Tech: Initiate Sell on Company, target price at Rs 1335/sh 

Emkay on Shree Cement: Maintain Reduce on Company, target price at Rs 29650/sh 

Emkay on Ramco Cement: Maintain Reduce on Company, target price at Rs 1080/sh 

HSBC on Bharti Airtel: Maintain Hold on Company, target price at Rs 1125/sh 

Citi on Titan: Maintain Neutral on Company, target price at Rs 3900/sh 

January 11, 2024 08:37
Stock Recommendations: - Expert meet takeaway: Cement sector - Emkay Global 

Key takeaways from the cement expert meeting include: 1) Industry demand is expected to grow 9-10% in FY24 and will likely clock in a 7-8% CAGR (200-250bps higher than the last decade) over the next few years with increased infra spends; 2) Consolidation theme is expected to be at the forefront over the next few years with two groups (UltraTech and Adani) expected to be more aggressive; 3) Organic capacity addition of more than 30mt p.a. would likely be challenging due to limited suppliers for equipment manufacturing and erection works; and 4) East and South regions should continue to see more volatility in pricing because of higher capacity additions and irrational behavior by a few players. Top picks: UltraTech and Ambuja.

January 11, 2024 08:37
Stock Recommendations: Morgan Stanley: Overweight rating for Bank of India, cites strong 4% QoQ gross loan growth

BANKINDIA: Morgan Stanley gives overweight call on Bank Of India, says gross loan growth was strong at 4% QoQ mainly driven by domestic loan growth at 5% QoQ.

January 11, 2024 08:36
Stock Recommendations: ZOMATO: HSBC maintains buy call on Zomato and says after an extremely strong 2023, can expect relatively muted business

.

January 11, 2024 08:35
Stock Recommendations: HSBC downgrades Bharti Airtel to hold, expects ARPU rise and market share gain

BHARTIARTL: HSBC downgrades Bharti Airtel to hold, says ARPU to rise, see market share gains in mobile. Growth in non-mobile & capex intensity to decline.

January 11, 2024 08:35
Stock Recommendations: Morgan Stanley: Initiates overweight on Coforge, prefers as top midcap IT pick

COFORGE: Morgan Stanley initiates overweight call on Coforge, says co is preferred Midcap IT pick. A focused & transformed Coforge has shown scalability attributes that can keep its revenue growth profile higher for longer vs peers.

January 11, 2024 08:34
Stock Market Live Today: New India Assurance confirms Rs 2379 crore GST demand, considers next steps

The New India Assurance Company Ltd (NIACL) has received an Order from the Office of the Commissioner of CSGT and Central Excise, Thane, confirming an earlier GST demand of Rs 2379.13 crore along with interest and penalty of Rs 237.91 crore.

“On receipt of the Order we have intimated the same to the Board in its meeting held on 10.01.2024. Based on the developments in the Industry, the company would be deciding further course of action,” the public sector general insurer said in a regulatory filing.

The public sector general insurer said it has received the Order in respect of the late-September 2023 SCN (show cause notice) confirming GST demand along with the interest and penalty. 

NIACL said it had received a SCN cum demand notice in late September 2023 from the Directorate General of GST Intelligence, Pune Zonal Unit, under the Central Goods and Services Tax Act, 2017, alleging a tax demand of 2379.13 crore (excluding interest and penalty).

The SCN cum demand notice was against: 1)non-payment of GST on the Co-insurance premium accepted as Follower in the co-insurance transaction from July 2017 to March 2022 amounting to Rs 1930.12 crore and 2) non-payment of GST on re-insurance commission booked/accepted on the reinsurance services ceded to various Indian and Foreign Re-Insurance Companies amounting to Rs 449 crore.

The non-life insurer emphasised that the alleged SCN pertains to matters relating to industry wide issues.

ends

January 11, 2024 08:22
Stock Recommendations: METRO BRANDS – TOP PICK 2024: Adding feet’s to the marathon: Motilal Oswal Financial 

(METROBRA IN, Mkt Cap USD4.1b, CMP INR1260, TP INR1530, 21% Upside, Buy)

The discretionary category has seen weak demand trends in the last few quarters, but METRO has continued to post industry-leading growth, led by steady footprint expansion. Although it could face moderation in SSSG and margin rebase in the near term, we expect the company to deliver secular 20% growth over the next five years, driven by its strong execution prowess evident from superlative store economics, a huge runway of footprint addition, excellent cashflow and ROIC profile, and new brand additions.

January 11, 2024 08:12
Stock Market Live Today: Stocks that will see action today: January 11, 2024

Buzzing Stocks: TCS, Infosys, HDFC Asset Management, 5Paisa, Sundram Fasteners, Bank of India, Kalyani Steel, New India Assurance, Heranba Industries, NBCC, CESC, Dhunseri Ventures, Phoenix Mills, Polycab India, Honda India Power, Metro Brands, Angel One, Safari Industries

January 11, 2024 08:11
Stock Market Live Today: Welspun Ent: Company pledges Rs 40,000 crore investment to set up green hydrogen, green ammonia ecosystem in Gujarat.

January 11, 2024 07:58
Stock Market Live Today: Ease My Trip: Company venturing into insurance sector with the launch of its subsidiary

January 11, 2024 07:58
Stock Market Live Today: KIMS: Company has acquired an additional 5.04% equity stake in Sarvejana Healthcare

January 11, 2024 07:57
Stock Market Live Today: Steel Strips: Directorate General Of Trade Remedies recommends anti-dumping duty on aluminum alloy road Wheel Imports from China for 5 years

January 11, 2024 07:56
Stock Market Live Today: NCC: Smallcap World Fund INC bought 36.15 Lakh Shares at 185.23/Share

January 11, 2024 07:44
Stock Market Live Today: Bank of Baroda hikes lending rates by 5 bps for overnight, six month & one year tenures w.e.f. January 12, 2024

January 11, 2024 07:44
Stock Market Live Today: Securities in F&O Ban For Trade Date 11-Jan-2024

• BALRAMCHIN 

• BANDHANBNK 

• CHAMBLFERT 

• ESCORTS 

• HINDCOPPER 

• IEX 

• INDUSTOWER

• INDIACEM 

• NATIONALUM 

• PEL

• PVRINOX

• SAIL

• ZEEL

January 11, 2024 07:20
Commodities Market Live Today: Copper: Initiate shorts on futures

Copper futures on the Multi Commodity Exchange (MCX) have been on a decline since the beginning of this month. It started to fall after facing resistance at ₹740.

January 11, 2024 07:19
Stock Market Live Today: Trading guide for January 11, 2024: Intraday supports, resistances for Nifty50 stocks

Here are the intraday supports and resistances for widely traded stocks such as Reliance Industries, ITC, ONGC, Infosys, HDFC Bank, TCS, and SBI

January 11, 2024 07:18
Stock Market Live Today: Stock to buy today: PNC Infratech (₹368.35): BUY

The short-term outlook is bullish for PNC Infratech surge 6 per cent on Wednesday. The stock well above the key intermediate resistance level of ₹362. The region around ₹360 will be a very strong resistance. 

January 11, 2024 07:11
Stock Recommendations: Goldman Sachs upgrades Shriram Finance, Maintains neutral on MMFS with TP adjustments

Goldman Sachs Upgrade Shriram Finance to Buy from Neutral; raise earnings estimates by 10%/13% for FY25E/FY26E; Increase 12-m TP by 43% to Rs2,758 (from Rs 1,926)

Goldman Sachs reiterate Neutral rating on MMFS; Cut EPS estimate by 22% in FY24, and by 8% in FY25E (to also reflect 2QFY24 earnings). Consequently, lower our TP by 9% to Rs 264 (from Rs 291)

January 11, 2024 07:10
Stock Market Live Today: Fund Flow Activity: 10 January 2024 (In Cr)

Turnover: (NSE + BSE)

Cash Volume: 96184.51 + 8210.2 Total: 104394.71

F&O Volume: 368993.72 + 292741.95 Total: 661735.67

Provisional Cash Rs. In Crs.

FII/FPI: NET SELL: -1721.35

(9621.87 - 11343.22) 

DII: NET BUY: +2080.01

(10461.84 - 8381.83)

January 11, 2024 07:10
Stock Market Live Today: Researchbytes Analyst App’s Recent Interview as of 18:31 p.m. Wednesday 10 January 2024

https://play.google.com/store/apps/details?id=com.intelegain.rb

BLS International Services : Shikhar Aggarwal, Joint Managing Director

Realisations Have Increased In Q3: BLS International Svcs

https://www.youtube.com/watch?v=XPGdoS9Og5g

Cupid: Aditya Halwasiya, MD

Cupid Limited: Fund Raise On The Anvil?

https://www.youtube.com/watch?v=_-v03k_RfcU

DLF: Ashok Tyagi, Whole Time Director

DLF Privana South Sold Out Within 72 Hours; Ashok Tyagi In Conversation With Nikunj Dalmia

https://www.youtube.com/watch?v=NFKDgXHh5fc

Delta Corp: Anil Malani, CFO

Slump In Revenue As Customers Had To Pay 28% GST Upfront On Chips: Delta Corp

https://www.youtube.com/watch?v=gD4zjli1kco

*Nuvama Wealth M: Prateek Parekh, *

Expect Moderation In Returns In 2024 Versus What Was Seen In 2023: Nuvama Group

https://www.youtube.com/watch?v=n9dYEM9QFBM

SBI: SK Ghosh, Group Chief Economic Advisor

K-shape Recovery Post-Pandemic Seems Best Flawed And Incomplete: SK Ghosh, SBI

https://www.youtube.com/watch?v=K2tNd_8sfww

Yatra.com: Dhruv Shringi, CEO

Lakshadweep Has Seen As 30x Increase In Searches Over The Last 4 Days: Yatra Online

https://www.youtube.com/watch?v=qURcS4m5QyY

Bajaj Auto: Dinesh Thapar, CFO

Buyback Decision Reflects Co’s Strong Business Performance: Bajaj Auto

https://www.youtube.com/watch?v=DSW8RXQkcNE

Brigade Ent: Pradyumna Krishna Kumar, Executive Director

Will Complete New Projects In Chennai Over Next 3-4 Years: Brigade Group

https://www.youtube.com/watch?v=xvP_-WFaQm4

Chambal Fert: Abhay Baijal, MD

If Red Sea Issue Flares Up Till March, This Will Be A Long-Term Impact: Chambal Fertilsers

https://www.youtube.com/watch?v=AAmzZYikeUw

Events today…

https://www.researchbytes.com/Default.aspx?cc=event

Results today…

https://www.researchbytes.com/Default.aspx?cc=result

Replays available at www.researchbytes.com

January 11, 2024 07:08
Stock Market Live Today: Major U.S. listed stocks result calendar 11.01.2024 

Taiwan Semiconductor Manufacturing Company (Tentative) (Sector- Technology)

January 11, 2024 07:08
Stock Market Live Today: Economic Calendar - 11.01.2024

19.00 U.S. CPI y/y (Expected: 3.2% versus Previous: 3.1%)

19.00 U.S. Unemployment Claims (Expected: 211K versus Previous: 202K)

January 11, 2024 07:03
Stock Market Live Today: Asian markets rise amid upbeat US stock trends

On Thursday, Asian stocks experienced an upswing, mirroring the positive momentum seen in US stocks ahead of the impending release of inflation data on Thursday. Japanese equities, particularly blue-chip stocks, opened higher on Thursday, riding on the surge in US tech shares and reaching their highest levels in over three decades. This increase in stock values can be attributed in part to the depreciation of the yen, which hit its lowest against the dollar in a month on Wednesday. 

In early trade, the Japanese Nikkei 225 index surged by 1.40%, or 483.27 points, reaching 34,924.99, while the broader Topix index rose by 1.47%, or 36.05 points, to 2,480.53. South Korea’s KOSPI also experienced a gain of 0.28%, or 7.15 points, reaching 2,549.13. Hong Kong’s Hang Seng index climbed by 0.35%, or 56.89 points, to 16,154.17, and Australia’s S&P/ASX 200 saw an increase of 35.80 points, or 0.48%, reaching 7,504.30.

Earlier on Wednesday, the Dow Jones Industrial Average rose by 170.57 points, or 0.45%, to 37,695.73. The S&P 500 recorded a gain of 26.95 points, or 0.57%, closing at 4,783.45, while the Nasdaq Composite advanced by 111.94 points, or 0.75%, reaching 14,969.65.

Published on January 11, 2024

Copyright© 2024, THG PUBLISHING PVT LTD. or its affiliated companies. All rights reserved.
'''

In [41]:
payload = {
    "inputs":  prompt,
    "parameters": parameters,
    "stream": True ## <-- to have response stream.
}

In [42]:
def get_realtime_response_stream(sagemaker_runtime, endpoint_name, payload):
    response_stream = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name,
        Body=json.dumps(payload), 
        ContentType="application/json",
        CustomAttributes='accept_eula=false'
    )
    return response_stream

In [44]:
smr = sess.boto_session.client("sagemaker-runtime")

In [51]:
import io
import json

class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line) + 1
                full_line = line[:-1].decode("utf-8")
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["token"]["text"]
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

In [53]:
# client = boto3.client("runtime.sagemaker")
start_time = time.time()
response = smr.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
)

for token in TokenIterator(response["Body"]):
    print(token, end="")


Terms of Use | Privacy Policy | FAQs

Powered by: Techtrend Technologies Pvt. Ltd.

http://www.thehindubusinessline.com/markets/stock-market-live-today/article65340786.ece

## Live Stock Prices from BSE and NSE. View Intraday Charts & Stock Quotes of BSE & NSE. Also check Share prices, Today’s Market Updates & Stock Market News of BSE & NSE.

Intraday Data delayed by at least 15 minutes or per exchange requirements.

Disclaimer: All efforts have been made to ensure accuracy of the data. However, we are not liable for any misbalances arising out of the data provided on this website. We advise you to independently verify the data before acting on it.

Please enter a valid email address.

Invalid Email Address. Please enter a valid email address.

Please select at least one option.

The above data is sourced from various sources and has not been verified by Moneycontrol. You can track commodity and equity prices in real-time on Moneycontrol.

By using this website, you acknowledge that y

In [105]:
import time

def query_stream_and_measure_time(query_id, payload):
    """
    Function to handle streaming and timing for a single query.
    query_function: The function to invoke the streaming endpoint.
    *args, **kwargs: Arguments for the query_function.
    Returns a tuple containing query_id, time to first word, and total time.
    """
    start_time = time.time()
    first_word_time = None
    total_time = None
    output = []
    response = smr.invoke_endpoint_with_response_stream(
                    EndpointName=endpoint_name,
                    Body=json.dumps(payload),
                    ContentType="application/json",
                )

    for line in TokenIterator(response["Body"]):
        output.append(line)
        if not first_word_time and line.strip():
            first_word_time = time.time()
    
    total_time = time.time()

    time_to_first_word = first_word_time - start_time if first_word_time else None
    total_time = total_time - start_time
    tps = len(output)/(total_time - time_to_first_word)
    return query_id, output, time_to_first_word, total_time, tps

In [78]:
len(payload['inputs'].split(' '))

8827

In [79]:
import numpy as np

In [81]:
np.random.choice(2031)

1392

In [85]:
new_len

array([757, 588, 103, ..., 533, 877,  40])

In [102]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent

In [103]:
from concurrent.futures import ThreadPoolExecutor

def run_parallel_queries(prompt, num_queries=20):
    """
    Run multiple queries in parallel and measure times.
    query_function: The function to invoke the streaming endpoint.
    num_queries: Number of parallel queries to run.
    *args, **kwargs: Arguments for the query_function.
    """
    results = []
    payloads = []
    words = prompt.split(' ')
    for i in range(20):
        new_len = np.random.randint(1024, 2048)
        new_start = np.random.randint(0, len(words) - new_len)
        new_prompt = ' '.join(words[new_start:(new_start+new_len)])
        payloads.append(new_prompt)
    payload = {
        "inputs":  prompt,
        "parameters": parameters,
        "stream": True ## <-- to have response stream.
    }

    with ThreadPoolExecutor(max_workers=num_queries) as executor:
        futures = [executor.submit(query_stream_and_measure_time, i, {
        "inputs":  payloads[i],
        "parameters": parameters,
        "stream": True ## <-- to have response stream.
    }) for i in range(num_queries)]
        for future in as_completed(futures):
            results.append(future.result())
    
    return results

In [106]:
res = run_parallel_queries(prompt, 5)

In [108]:
[(x[0], len(x[1]), x[2], x[3], x[4]) for x in res]

[(1, 271, 1.1775479316711426, 17.07368302345276, 17.048169157804175),
 (4, 288, 1.941161870956421, 17.9459228515625, 17.99464549011302),
 (3, 318, 4.983713865280151, 21.511450052261353, 19.240384551302718),
 (0, 471, 3.3899338245391846, 28.476318836212158, 18.775124426290933),
 (2, 788, 6.656641721725464, 46.89753985404968, 19.582067910333866)]

In [110]:
res8 = run_parallel_queries(prompt, 8)

In [111]:
[(x[0], len(x[1]), x[2], x[3], x[4]) for x in res8]

[(6, 24, 6.823728799819946, 8.413669109344482, 15.094906303229134),
 (2, 29, 9.885974884033203, 12.025516033172607, 13.55430813362238),
 (5, 114, 8.218945980072021, 15.1265389919281, 16.50357799081855),
 (7, 174, 3.7338621616363525, 15.307170152664185, 15.034595133465118),
 (0, 185, 5.19197416305542, 16.702793836593628, 16.071835477128495),
 (4, 121, 11.666991233825684, 17.283578157424927, 21.54333256939257),
 (1, 421, 1.9467689990997314, 25.723637104034424, 17.706284870740607),
 (3, 555, 1.375389814376831, 31.633113145828247, 18.34242430999773)]

In [74]:
query_stream_and_measure_time(1, payload)

(1,
 ['\n',
  'For',
  ' any',
  ' query',
  ',',
  ' please',
  ' contact',
  ' us',
  ' at',
  ' -',
  ' [',
  'email',
  ' protected',
  ']',
  ' or',
  ' call',
  ' us',
  ' at',
  ' ',
  '9',
  '3',
  '2',
  '3',
  '7',
  '7',
  '2',
  '8',
  '9',
  '7',
  '.',
  '\n',
  '\n',
  '©',
  ' ',
  '2',
  '0',
  '2',
  '4',
  ' The',
  ' Hindu',
  ' Business',
  'Line',
  '.',
  ' All',
  ' rights',
  ' reserved',
  '.',
  ' For',
  ' re',
  'print',
  ' rights',
  ',',
  ' contact',
  ' [',
  'email',
  ' protected',
  ']',
  '\n',
  '\n',
  'For',
  ' subscription',
  ',',
  ' contact',
  ' [',
  'email',
  ' protected',
  ']',
  ' or',
  ' call',
  ' us',
  ' at',
  ' ',
  '9',
  '3',
  '2',
  '3',
  '7',
  '7',
  '2',
  '8',
  '9',
  '7',
  '.',
  '\n',
  '\n',
  '###'],
 0.9410438537597656,
 4.789793968200684,
 22.344916516486453)

In [69]:
import time

# Start the timer before making the request
start_time = time.time()

# Invoke the streaming endpoint
response = smr.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
)

first_word_time = None
output = []

# Iterate over the response stream to process the data
for line in TokenIterator(response["Body"]):
    # Assuming each line is a text and you're collecting all lines
    output.append(line)
    print(line, end="")  # Print each line as it's received

    # Capture the time when the first word is received, if not already done
    if not first_word_time and line.strip():
        first_word_time = time.time()

# End time after all words are received
end_time = time.time()

# Calculate time differences
time_to_first_word = first_word_time - start_time
total_time = end_time - start_time
print("")
print(f"Time taken to receive first word: {time_to_first_word} seconds")
print(f"Total time to receive all words: {total_time} seconds")



For any feedback, complaints, or suggestions, write to the editor at [email protected]

Except for the editorially provided content, this page is generated automatically from a feed of the associated press.

Courtesy: Business Standard

Read More: Stock Market<|im_end|>
Time taken to receive first word: 0.9630029201507568 seconds
Total time to receive all words: 3.5161631107330322 seconds


In [71]:
len(output)

59

In [70]:
full_output = ''.join(output)

In [59]:
output = ''.join(output)

In [72]:
len(tokenizer(output)['input_ids'])

59

In [63]:
431/19

22.68421052631579

In [55]:
time_to_first_word

1.3371129035949707

In [48]:
resp = get_realtime_response_stream(smr, endpoint_name, payload)

In [49]:
print(resp)

{'ResponseMetadata': {'RequestId': 'd3a05227-802e-4409-8dfb-3578f75f73be', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd3a05227-802e-4409-8dfb-3578f75f73be', 'x-amzn-invoked-production-variant': 'AllTraffic', 'x-amzn-sagemaker-content-type': 'text/event-stream', 'date': 'Thu, 11 Jan 2024 07:31:36 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'text/event-stream', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.eventstream.EventStream object at 0x17c32bd90>}


In [ ]:
%%writefile serving.properties
engine=MPI 
option.model_id=teknium/OpenHermes-2.5-Mistral-7B
option.trust_remote_code=true
option.tensor_parallel_degree=1
option.max_rolling_batch_size=32
option.rolling_batch=auto
option.output_formatter=jsonlines
option.paged_attention=true
option.enable_streaming=true

In [2]:
from sagemaker.huggingface.model import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/username/Library/Application Support/sagemaker/config.yaml


In [5]:
hub = {
  'HF_MODEL_ID':'teknium/OpenHermes-2.5-Mistral-7B', # model_id from hf.co/models
  'HF_TASK':'text-generation'                           # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                                                # configuration for loading model from Hub
   role=role,                                              # IAM role with permissions to create an endpoint
   transformers_version="4.28",                             # Transformers version used
   pytorch_version="2.0",                                  # PyTorch version used
   py_version='py310',                                      # Python version used
)

In [6]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g5.2xlarge"
)

--------!

In [7]:

# example request: you always need to define "inputs"
data = {
"inputs": {
	"question": "What is used for inference?",
	"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}